## Preliminary Web Scraping

  * Collecting stats names and stat IDs

  * Getting some practice scraping data from an API and using json files

In [36]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
import os
import lxml.html as lh
import requests as req
import urllib
import json
from itertools import chain
import sqlite3 as db
import time

We will scrape data from the PGA Tour website using their api. The api url can be found by inspecting the tables we want (from https://www.pgatour.com/players/player.28237.rory-mcilroy.html for Rory McIlroy) and using the Network tab to view from where the website collects it's data.

In [37]:
url = 'https://statdata-api-prod.pgatour.com/api/clientfile/YtdPlayerStatsArchive?P_ID=28237&YEAR=2021&format=json&userTrackingId=exp=1629169369~acl=*~hmac=2e4932d30fe2ebab841a2596f1387714ab37d4c52729620410b3425117fe64fb'

In [38]:
api_id = {'McIlroy' : '28237'}
player_id = api_id['McIlroy']
year = ''
hmac = ''
tracking_id = ''
url_format = 'https://statdata-api-prod.pgatour.com/api/clientfile/YtdPlayerStatsArchive?P_ID=' + player_id + '&YEAR=' + year + '&format=json&userTrackingId=exp=' + tracking_id + '~acl=*~hmac=' + hmac

The player_id and year are easy to choose. But the hmac and tracking_id change every time I open the webpage. I need to find a way to automate the retrieval of the hmac and tracking_id for each webpage.

Note: this url has to be updated frequently. Potentially every time you run this file you may need to update the url

In [39]:
request = req.get(url)
j = request.json()

In [40]:
recap_data = j.get('plrs')[0]#.get('years')[0].get('tours')[0].get('statCats')[0].get('stats')

In [41]:
off_the_tee_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[1].get('stats')

In [42]:
app_the_green_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[2].get('stats')

In [43]:
ar_the_green_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[3].get('stats')

In [44]:
putting_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[4].get('stats')

In [45]:
scoring_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[5].get('stats')

In [46]:
streaks_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[6].get('stats')

In [47]:
recap = pd.DataFrame(recap_data)

In [48]:
recap['years']

0    {'year': '2021', 'disclaimer': 'Player did not...
Name: years, dtype: object

We nearly have everything we need, our problem is in the additionals column. Notice how our stats query is a list of dictionaries with embedded dictionaries for the 'additionals' key. The other columns converted to the data frame nicely because they did not have embedded dictionaries within them. 

Now we have a separate data frame for the additional statistics column from our recap data frame. Now we can do this for all of our categories.

In [49]:
off_the_tee = pd.DataFrame(off_the_tee_data)

In [50]:
off_the_tee.head()

,statID,name,value,rank,projRank,additionals
0,02674,SG: Tee-to-Green,1.382,8,,"[{'title': 'SG:OTT', 'value': '.702'}, {'title..."
1,02567,SG: Off-the-Tee,.702,5,,"[{'title': 'Total SG:OTT', 'value': '35.804'},..."
2,101,Driving Distance,319.0,2,,"[{'title': 'Total Distance', 'value': '39,552'..."
3,317,Driving Distance - All Drives,308.2,2,,"[{'title': 'Total Distance', 'value': '218,790..."
4,159,Longest Drives,379,87,,"[{'title': 'Tournament', 'value': 'the Memoria..."


In [51]:
off_the_tee_additionals = pd.DataFrame(list(chain.from_iterable(off_the_tee['additionals'])))

In [52]:
app_the_green = pd.DataFrame(app_the_green_data)

In [53]:
ar_the_green = pd.DataFrame(ar_the_green_data)

In [54]:
putting = pd.DataFrame(putting_data)

In [55]:
scoring = pd.DataFrame(scoring_data)

In [56]:
streaks = pd.DataFrame(streaks_data)

In [57]:
app_the_green_additionals = pd.DataFrame(list(chain.from_iterable(app_the_green['additionals'])))

In [58]:
ar_the_green_additionals = pd.DataFrame(list(chain.from_iterable(ar_the_green['additionals'])))

In [59]:
putting_additionals = pd.DataFrame(list(chain.from_iterable(putting['additionals'])))

In [60]:
scoring_additionals = pd.DataFrame(list(chain.from_iterable(scoring['additionals'])))

In [61]:
streaks_additionals = pd.DataFrame(list(chain.from_iterable(streaks['additionals'])))

In future iterations I will make a function that automates this. But for now this will suffice.

Let's now concatenate the dataframes, drop the additionals column from the primary data, and drop the projRank column. Let's also remove duplicate rows from both.

In [62]:
McIlroy_data = pd.concat([recap, off_the_tee, app_the_green, ar_the_green, putting, scoring, streaks], axis = 0)
McIlroy_data = McIlroy_data.drop(['projRank', 'additionals'], axis = 1)
McIlroy_data = McIlroy_data.drop_duplicates('statID')

In [63]:
McIlroy_additionals = pd.concat([off_the_tee_additionals, #recap_additionals
                                 app_the_green_additionals, ar_the_green_additionals,
                                 putting_additionals, scoring_additionals,
                                 streaks_additionals], axis = 0)
McIlroy_additionals = McIlroy_additionals.drop_duplicates('title')

In [64]:
McIlroy_data_transposed = McIlroy_data.transpose()
#McIlroy_data_transposed.rename(columns = McIlroy_data_transposed[2,:])
McIlroy_data_transposed.columns = McIlroy_data_transposed.iloc[1, :]

In [65]:
McIlroy_additionals_transposed = McIlroy_additionals.transpose()
McIlroy_additionals_transposed.columns = McIlroy_additionals_transposed.iloc[0,:]


In [66]:
McIlroy_data.loc[McIlroy_data['statID'] == '103']

,plrNum,plrName,years,statID,name,value,rank
1,NaN,NaN,NaN,103,Greens in Regulation Percentage,65.86%,99


In [67]:
stat_ids = McIlroy_data['statID'].values

In [68]:
stat_names = McIlroy_data['name'].values

In [69]:
stats = {stat_ids[i] : stat_names[i] for i in range(1, len(stat_ids))}

Since we have the stats names and the stats ids we can make a dataframe and then migrate it onto a database so we never have to run the previous processes again. It's very clunky and there are certainly better ways to do what I have just done, but for the purposes of this post and to be in the spirit of learning I decided to play around and try to learn some new tools and file types. In the future, I probably don't have to use an API to scrape the stat names and stat ids, nor would I have to unwind the json response data and create individual data frames for each stat category then concatenate them after I wrangle them into the same dimensions. But for now, I have the basics for what I need to scrape every statistic on the PGA Tour's website.

In [70]:
data = {'stat_ids' : stat_ids[1:], 'stat_names': stat_names[1:]}
stat_ids_df = pd.DataFrame.from_dict(data)
conn = db.connect('stat_ids.db')
stat_ids_df.to_sql('stat_ids', conn, if_exists = 'replace')
conn.close()

## Collecting all the data

  * Now that we have the stats database, we can construct dataframes for each statistic we want in each year we want, and make a master database!

  * We can also **automate** the retrieval of all of the statistics and the construction of the master database.

In [71]:
conn = db.connect('stat_ids.db')

c = conn.cursor()

query = conn.execute("SELECT * From stat_ids")
cols = [column[0] for column in query.description]
stat_ids = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

conn.close()

In [72]:
stat_ids.head()

,index,stat_ids,stat_names
0,0,02674,SG: Tee-to-Green
1,1,02567,SG: Off-the-Tee
2,2,101,Driving Distance
3,3,317,Driving Distance - All Drives
4,4,159,Longest Drives


In [73]:
stat_nums = stat_ids['stat_ids']
stat_names = stat_ids['stat_names']
stat_dict = dict(zip(stat_nums, stat_names))

I included this extra dictionary as practice on how to create dictionaries from a database/dataframe. It may seem redundant but I wanted the extra practice.

### Automating the data retrieval

We're going to need to write some functions to help us scrape the PGA Tour's website and assemble our database. These are:

  1. A function to generate urls for each statistic's webpage for a given season.

  2. A function to extract the headers of the data table in the urls.

  3. A function to scrape the data based on the extracted headers.

  4. A function to take the extracted data and assemble them into dataframes.

  5. A function to execute the above three functions and concatenate all the data frames into one database.

In [74]:
def get_links(stat_ids, year):
    """
    Create a list of links for each statistic in a given year.
    """
    base_url = 'https://www.pgatour.com/content/pgatour/stats/stat.'
    urls = [base_url + key + '.y' + str(year) + '.html' for key in stat_dict.keys()]
    return urls

Now what do I need? I know I can access all of the meaning headers I want. I need to change some of the headers, for example headers that just say '%', for convenience, then I can rework my existing scraping functions to use the headers I have here. This will also make my make_data_frame function much cleaner and simpler to use as I won't have many separate cases involved.

In [88]:
def get_headers(links):
    headers_nested_list = []
    stat_dict_keys = [key for key in stat_dict.keys()]

    for i in range(len(links)):
        headers = []
        stat_name = stat_dict[stat_dict_keys[i]].upper()
        response = req.get(links[i])
        soup = BeautifulSoup(response.content, 'lxml')

        #Get rounds header
        rank_this_week = soup.find_all(class_="hidden-small hidden-medium")[0].get_text()
        headers.append('RANK ' + rank_this_week)

        player_name = soup.find_all(class_ = 'player-name')[0].get_text()
        headers.append(player_name)

        streak_stats = ['CONSECUTIVE CUTS', 'YTD PAR OR BETTER STREAK',
                       'BEST YTD 1-PUTT OR BETTER STREAK','BEST YTD STREAK W/O A 3-PUTT',
                       'CONSECUTIVE SAND SAVES', 'CONSECUTIVE FAIRWAYS HIT',
                       'CONSECUTIVE GIR', 'CONSECUTIVE HOLES BELOW PAR',
                       'CONSECUTIVE PAR 3 BIRDIES', 'CONSECUTIVE BIRDIES STREAK',
                       'CONSECUTIVE BIRDIES/EAGLES STREAK',
                       'CURRENT STREAK WITHOUT A 3-PUTT']
            
        #Get other headers
        stat_headers = soup.find_all(class_="col-stat")
        for header in stat_headers:
                headers.append(header.get_text())

        headers.insert(1, 'RANK LAST WEEK')
        # All edge cases:

        if len(headers) > 4 and headers[4] == 'TOTAL PUTTS':
            if len(headers) == 7:
                #print('edge case 1')
                headers.insert(3, 'ROUNDS')
                headers[4] = stat_name
                #print('edge case 1')
                #print(headers)
                headers_nested_list.append(headers)
                #print(links[i])
                #print(stat_name)
                continue
            else:
                headers[3] = stat_name
                #print('edge case 2')
                #print(headers)
                headers_nested_list.append(headers)
                #print(links[i])
                #print(stat_name)
                continue

        if stat_name in streak_stats:
            #print('edge case 3')
            headers[3] = stat_name
            #print(headers)
            headers_nested_list.append(headers)
            #print(links[i])
            #print(stat_name)
            continue
            
        elif len(headers) > 6 and headers[5] == 'TOTAL ADJUSTMENT':
            #another edge case
            #print('edge case 4')
            headers.insert(3, 'ROUNDS')
            headers[4] = stat_name
            #print(stat_name)
            #print(headers)
            headers_nested_list.append(headers)
        elif len(headers) == 6 and headers[5] == 'TOTAL ROUNDS':
            if headers[4] == 'TOTAL STROKES':
                if stat_name == 'SCORING AVERAGE (ACTUAL)':
                    #print('edge case 5')
                    headers.insert(3, 'ROUNDS')
                    headers[4] = stat_name
                    #print(stat_name)
                    #print(headers[4])
                    headers_nested_list.append(headers)
                    #print(headers)
                    #print(links[i])
                    #print(stat_name)
                    continue
            if headers[4] == '# OF BIRDIES':
                #print('edge case 6')
                headers.insert(3, 'ROUNDS')
                headers[4] = stat_name
                #print(stat_name)
                #print(headers[4])
                headers_nested_list.append(headers)
                #print(headers)
                #print(links[i])
                #print(stat_name)
                continue
            #print(headers[3])
            #print('edge case 7')
            headers[3] = stat_name
            #print(headers[3])
            #print(stat_name)
            #print(headers)
            headers_nested_list.append(headers)

        else:
            #print(headers)
            if len(headers) == 3:
                #print("edge case 8")
                headers.insert(3, 'ROUNDS')
                headers.insert(4, stat_name)
                headers_nested_list.append(headers)
            
            else:
                #print("edge case 9")
                headers.insert(3, 'ROUNDS')
                headers[4] = stat_name
                headers_nested_list.append(headers)

        #print(headers)
        #print(links[i])
        #print(stat_name)
    return headers_nested_list

The commented out print statements are for debugging purposes. I mainly develop in a jupyter environment and thus far I have not found a debugger similar to what IDE's have. So for now, my code looks a bit cluttered. It would also be beneficial to implement unit testing and try-except blocks.

In [76]:
def get_stats(link):
    '''
    Collect the stats from a soup object and return as a list.
    '''
    response =  ''
    # Implement a sleep between failed attempts to access the webpage.
    
    while response == '':
        try:
            response = req.get(link)
            break
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            print("ZZzzzz...")
            time.sleep(5)
            print("Was a nice sleep, now let me continue...")
            continue

    # access the data
    soup = BeautifulSoup(response.content, 'lxml')
    # find the datatable
    table = soup.find('table', attrs = {'id': 'statsTable'})
    stats = []
    # find the body of the datatable
    body = table.find('tbody')
    # find the rows of the datatable
    all_rows = body.findAll('tr')
    # go through each row and append each column of each row 
    # to our stats list. 
    for row in all_rows:
        line = []
        for column in row.findAll('td'):
            data = column
            data = data.text
            line.append(data)
        stats.append(line)
    return stats

In [77]:
def make_data_frames(links, year):
    """
    This function takes a scoring category and a provided url_list,
    calls the get_datas function to scrape the data, and creates a
    data frame for every url in the url list and appropriately
    renames the columns based upon the category provided. This returns
    a list of data frames.
    """
    
    headers_list = get_headers(links)
    df_list = []

    for i in range(len(links)):
        data = get_stats(links[i])
        df = pd.DataFrame(data, columns = headers_list[i])
        # remove \n characters and whitespace
        df['RANK THIS WEEK'] = df['RANK THIS WEEK'].str.replace('\n', '')
        df['RANK LAST WEEK'] = df['RANK LAST WEEK'].str.replace('\n', '')
        df['PLAYER NAME'] = df['PLAYER NAME'].str.replace('\n', '').str.strip()
        df_list.append(df)
    return df_list

Create a list of year numbers from 2010 to 2021 for the seasons to extract.

In [81]:
years = [str(i) for i in range(2010, 2022)]

Create a list of year numbers from 2010 to 2021 for the seasons to extract.

In [90]:
def make_master_data_frame():
    for year in years:
        print(year)
        # create the list of links for each statistic with the current year.
        links = get_links(stat_ids, year)
        # create the associated dataframes with each link in the current year.
        data_frames = make_data_frames(links, year)
        # remove duplicate columns. Perhaps in the future I will 
        # change the name of each column slightly in the body of the
        # make_data_frames function but for now I don't see many
        # uses for these statistics.
        duplicate_columns = ['ROUNDS','ATTEMTPS','RELATIVE TO PAR',
                            'MEASURED ROUNDS','TOTAL DISTANCE (FEET)',
                            '# OF ATTEMPTS', 'SUCCESSES','TOTAL PUTTS',
                            'TOTAL ROUNDS','PUTTS MADE','TOTAL HOLES',
                            'ROUND','HOLE','CURRENT STREAK',
                            'RANK THIS WEEK','RANK LAST WEEK',
                            'ATTEMPTS', 'TOTAL STROKES', 'TOTAL DISTANCE',
                            'YEAR/TOURN#', 'TOURNAMENT', 'POSSIBLE FWYS', 'GREENS HIT',
                            '# HOLES', 'GREENS HIT', 'RTP-NOT GOING FOR THE GRN',
                            'RTP-SUCCESSES', 'GIR RANK', '# OF HOLES',
                            'TOTAL BIRDIES', 'SG: OFF-THE-TEE','SG: APPROACH THE GREEN',
                            'SG: AROUND-THE-GREEN']

        columns_to_drop = []
        df_one = data_frames[0]
        for column in df_one.columns:
            if column in duplicate_columns:
                columns_to_drop.append(column)
        df_one = df_one.drop(columns_to_drop, axis = 1)

        # merge all Dataframes together

        for i in range(1, len(data_frames)):
            columns_to_drop = []
            for column in data_frames[i].columns:
                if column in duplicate_columns:
                    columns_to_drop.append(column)
            df_one = pd.merge(df_one, data_frames[i].drop(columns_to_drop, axis = 1), on ='PLAYER NAME', how = 'left')

        # add year column
        df_one['YEAR'] = year
        
        #Concat dataframe to overall dataframe    
        if year == '2010':
            df_total = pd.DataFrame()
            df_total = pd.concat([df_total, df_one], axis=0)
        else:
            df_total = pd.concat([df_total, df_one], axis=0)

    print('Finished Constructing Table')
    print(df_total.shape)
    return df_total

In [91]:
df_total = make_master_data_frame()
df_total.head()

2010


C:\Users\zytow\AppData\Local\Temp/ipykernel_22092/1279410816.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_one['YEAR'] = year


2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
Finished Constructing Table
(10462, 188)


,PLAYER NAME,SG: TEE-TO-GREEN,SG:OTT,SG:APR,SG:ARG,TOTAL SG:OTT,DRIVING DISTANCE,TOTAL DRIVES,DRIVING DISTANCE - ALL DRIVES,# OF DRIVES,...,BEST YTD STREAK W/O A 3-PUTT,CURRENT STREAK WITHOUT A 3-PUTT,CONSECUTIVE SAND SAVES,CONSECUTIVE FAIRWAYS HIT,CONSECUTIVE GIR,CONSECUTIVE HOLES BELOW PAR,CONSECUTIVE PAR 3 BIRDIES,CONSECUTIVE BIRDIES STREAK,CONSECUTIVE BIRDIES/EAGLES STREAK,YEAR
0,Adam Scott,1.609,.472,1.164,-.027,24.523,294.4,136,285.7,724,...,116,26,4,19,15,3,4,3,3,2010
1,Vijay Singh,1.494,.369,.979,.145,21.050,292.3,148,287.0,798,...,118,6,6,14,17,4,2,4,3,2010
2,Steve Stricker,1.383,.191,.773,.419,10.669,282.9,144,279.1,784,...,223,58,8,24,11,4,2,4,2,2010
3,Jim Furyk,1.159,.150,.641,.367,9.018,276.0,148,273.0,836,...,124,17,9,19,18,4,3,4,3,2010
4,Phil Mickelson,1.151,.185,.738,.228,10.556,299.1,152,295.0,798,...,154,7,6,12,18,6,2,6,3,2010


Load the dataframe onto a database for use in other projects.

In [92]:
conn = db.connect('pga_database.db')
df_total.to_sql('pga_stats_table', conn, if_exists = 'replace')
conn.close()

C:\Users\zytow\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:2869: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
C:\Users\zytow\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:900: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  temp.reset_index(inplace=True)


Access the database and create a dataframe from it.

In [93]:
conn = db.connect('pga_database.db')
query = conn.execute("SELECT * From pga_stats_table")
cols = [column[0] for column in query.description]
df_total = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [94]:
df_total.loc[df_total['PLAYER NAME'] == 'Rory McIlroy']

,index,PLAYER NAME,SG: TEE-TO-GREEN,SG:OTT,SG:APR,SG:ARG,TOTAL SG:OTT,DRIVING DISTANCE,TOTAL DRIVES,DRIVING DISTANCE - ALL DRIVES,...,BEST YTD STREAK W/O A 3-PUTT,CURRENT STREAK WITHOUT A 3-PUTT,CONSECUTIVE SAND SAVES,CONSECUTIVE FAIRWAYS HIT,CONSECUTIVE GIR,CONSECUTIVE HOLES BELOW PAR,CONSECUTIVE PAR 3 BIRDIES,CONSECUTIVE BIRDIES STREAK,CONSECUTIVE BIRDIES/EAGLES STREAK,YEAR
12,12,Rory McIlroy,.944,.698,.271,-.025,27.933,300.0,104,293.3,...,194,12,6,16,17,4,1,3,4,2010
378,0,Rory McIlroy,2.347,1.072,1.002,.273,42.860,310.1,106,298.6,...,110,97,10,12,13,4,4,4,2,2012
579,10,Rory McIlroy,1.077,.624,.420,.032,25.601,302.2,110,294.8,...,126,49,5,9,13,4,2,4,2,2013
749,0,Rory McIlroy,1.992,1.367,.602,.022,65.623,310.5,120,301.5,...,152,8,5,11,22,5,2,5,3,2014
1111,1,Rory McIlroy,1.813,1.192,.428,.193,52.438,306.2,108,300.7,...,106,64,4,15,21,4,2,4,4,2016
1305,7,Rory McIlroy,1.382,.965,.322,.095,34.731,317.2,88,309.9,...,85,4,16,8,17,3,2,3,3,2017
1497,5,Rory McIlroy,1.351,.761,.269,.322,38.798,319.7,122,305.7,...,184,50,7,14,26,6,2,6,1,2018
5780,0,Rory McIlroy,2.126,1.195,.633,.297,68.124,313.5,128,304.2,...,190,140,6,17,15,4,3,4,2,2019
5973,5,Rory McIlroy,1.314,.702,.504,.108,35.796,314.0,104,303.4,...,245,18,5,11,13,4,3,4,3,2020
10263,7,Rory McIlroy,1.382,.702,.588,.091,35.804,319.0,124,308.2,...,125,55,8,10,15,5,3,5,2,2021


Now I can access historical data for each player that the PGA Tour tracked from 2010 to 2021

## Now I can put this onto my website